<br><br><br>
<center> <h3 style="color:blue">========================================================</h3> </center>
<center> <h3 style="color:green">In the Name of Allah, the Most Beneficent, the Most Merciful</h3> </center>
<center> <h3 style="color:green">AI 2024 Online Summer Internship</h3> </center>
<center> <h3 style="color:green">Make Your Place in Top 20% AI Engineers of the World</h3> </center>
<center> <h3 style="color:blue">========================================================</h3> </center>
<br><br><br>

        *----------------------------- AUTHOR_DETAILS -------------------------------*
        |                                                                            |
        |        Project Title  = Multi-Label Course Classification                  |
        |                            (Fine-Tuning Approach)                          |
        |                                                                            |
        |        Author         = H. M. Adil Javaid                                  |
        |                                                                            |
        |        Copyright      = Copyright (C) 2024 H. M. Adil Javaid               |
        |                                                                            |
        |        License        = Public Domain                                      |
        |                                                                            |
        |        Version        = 1.0                                                |
        |                                                                            |
        *----------------------------------------------------------------------------*

<br><br><br>
<center> <h2 style="color:green">-------------------- PROJECT PURPOSE --------------------</h2> </center>
<br>
<center><h3>
The main purpose of this project is to demonstrate how the Multi-label Courses Classification problem can be treated as a Supervised Machine Learning problem using Python and the Mistral-7b Model.
</h3></center>
<br>
<center><h3>For this purpose, In Sha Allah, we will execute the Fine Tuning Approach.</h3></center>
<br>
<center> <h2 style="color:green">-------------------------------------------------------------------------</h2> </center>
<br><br><br>


# Step 1: Install Libraries

In [1]:
###We will update and install the necessary Python libraries.

##%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 12.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 73.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
Note: you may need to restart the kernel to use update

# Step 2:  Save Hugging Face and Weights and Biases API keys and access them

In [2]:
####save Hugging Face and Weights and Biases API keys and access them in the Kaggle notebook.

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Hugging_Face_Write")
secret_value_1 = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_value_2 = user_secrets.get_secret("wandb")

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

2024-08-21 09:07:40.209062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 09:07:40.209198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 09:07:40.333093: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
###  We will use the Hugging Face API to save and push the model to the Hugging Face Hub.
!huggingface-cli login --token $secret_value_0

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Step 3: Login to WanDB

In [5]:
wandb.login(key = secret_value_2)
run = wandb.init(
    project='Fine tuning mistral 7B', 
    job_type="training", 
    anonymous="allow"
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: javaidadil835 (javaidadil835-comsats-university-islamabad). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.4
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240821_090755-91gzo9lc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run atomic-snowflake-3
wandb: ⭐️ View project at https://wandb.ai/javaidadil835-comsats-university-islamabad/Fine%20tuning%20mistral%207B
wandb: 🚀 View run at https://wandb.ai/javaidadil835-coms

# Step 4: Base Model, Dataset, and New Model

In [6]:
base_model = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
dataset_name = "/kaggle/input/abstract-classification/Train_Sample.csv"
new_model = "mistral_7b_mlec_eng_1"

# Step 5: Binary to Text Conversion

In [7]:
## function to convert binary label into text labels
def convert_binary_to_label(data,courses_labels):
  
    """Converts binary emotion values to corresponding labels.
      Args:
        data: A list of lists, where the elements are binary values representing presence of emotion in a tweet.

      Returns:
        A list of lists, where each list contains the Abstract and a list of course labels.
    """

    result = []
    for row in data:
            abstract = row[0]
            courses = [label for label, value in zip(courses_labels, row[1:]) if int(value) == 1]
            if not courses:
                  courses.append('neutral')  # Add 'neutral' if the list is empty
            text_string = ', '.join(courses)
            result.append((text_string))

    return result

# Step 6: Convert Data into Mistral Train Data Format

In [8]:
##### function to convert data into mistral train data format

def convert_mistral_promt(my_df):
    promtlist = []
    for index, row in my_df.iterrows():
        promt= f"""\' <s>[INST] Being an expert who can detect courses
        present in a given text enclosed in “” please read the provided Urdu text. 
        Identify and list up to four prominent courses
        present in the provided Urdu text. 
        Consider the following emotion labels: 
        Computer
        Physics
        Mathematics
        Statistics
        Quantitative Biology
        Quantitative Finance
        neutral. 
    
        If no courses are detected or the text is unclear, return ['neutral']. 
        your answer should be in the form of a Python list containing emotion labels only mentioned above and presented like as an example
        given below:
        Emotion : ['Statistics', 'Mathematics']

        The text is : "{row['ABSTRACT']}" [/INS] "{row['Courses']}" </s>\' """
        promtlist.append(promt)

    # Create a DataFrame
    return promtlist

# Step 7: Load Libraries

In [9]:
import csv
import pandas as pd
import numpy as np
import random
from datasets import Dataset, DatasetDict
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split

# Step 8: Data Preparation and Splitting for Multi-label Course Classification

In [10]:
test_labels = ["Computer", "Physics", "Mathematics", "Statistics", "Quantitative Biology", "Quantitative Finance"]

# Seed for reproducibility
random.seed(0)

# Load data
with open('/kaggle/input/abstract-classification/Train_Sample.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile, delimiter=','))
    header_row = data.pop(0)

# Shuffle data
random.shuffle(data)

# Reshape and convert labels to integers
text, labels = list(zip(*[(f'{row[2].strip()}', [int(row[i]) for i in range(3, 9)]) for row in data]))

# Convert labels to numpy array
labels = np.array(labels, dtype=int)

# Create label weights
label_weights = 1 - labels.sum(axis=0) / labels.sum()

# Stratified train-test split for multilabel dataset
row_ids = np.arange(len(labels))
train_idx, y_train, val_idx, y_val = iterative_train_test_split(row_ids[:, np.newaxis], labels, test_size=0.1)
x_train = [text[i] for i in train_idx.flatten()]
x_val = [text[i] for i in val_idx.flatten()]

# Assuming `convert_binary_to_label` is a function that converts binary labels to actual label names
train_act_lab = convert_binary_to_label(y_train, test_labels)
val_act_lab = convert_binary_to_label(y_val, test_labels)

# Create dataframes for training and validation
train_df = pd.DataFrame({'ABSTRACT': x_train, 'Courses': train_act_lab})
val_df = pd.DataFrame({'ABSTRACT': x_val, 'Courses': val_act_lab})

# Assuming `convert_mistral_promt` is a function that converts dataframes to a suitable format for training
train_prompt = convert_mistral_promt(train_df)
val_prompt = convert_mistral_promt(val_df)

# Convert lists to datasets
train_dataset = Dataset.from_dict({'text': train_prompt})
val_dataset = Dataset.from_dict({'text': val_prompt})


# Step 9: Loading Mistral 7B Model

In [11]:
#################################
###Loading the Mistral 7B model
#################################

bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        #load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# Step 10: Loading Tokenizer

In [12]:
##Loading the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

# Step 11: Add Adopter to the Layer

In [13]:
####Adding the adopter to the layer
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

# Step 12: Parameters

In [14]:
#####Hyperparameters

training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    logging_dir="./logs",
    metric_for_best_model="eval_loss",
    learning_rate=1e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
##############SFT parameters

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

# Step 13: Run Trainer

In [16]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=60, training_loss=1.318112357457479, metrics={'train_runtime': 749.8059, 'train_samples_per_second': 0.3, 'train_steps_per_second': 0.08, 'total_flos': 4632754782633984.0, 'train_loss': 1.318112357457479, 'epoch': 5.0})

In [17]:
new_model = "mistral_7b_mlec_eng_1"
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:               eval/loss █▁▁▂▄
wandb:            eval/runtime ▆▂▂▁█
wandb: eval/samples_per_second ▄▄▄█▁
wandb:   eval/steps_per_second ▁▁▁▁▁
wandb:             train/epoch ▁▃▃▅▆▇██
wandb:       train/global_step ▁▃▃▅▆▇██
wandb:         train/grad_norm ▁█
wandb:     train/learning_rate ▁▁
wandb:              train/loss █▁
wandb: 
wandb: Run summary:
wandb:                eval/loss 1.36313
wandb:             eval/runtime 4.6625
wandb:  eval/samples_per_second 1.072
wandb:    eval/steps_per_second 0.429
wandb:               total_flos 4632754782633984.0
wandb:              train/epoch 5.0
wandb:        train/global_step 60
wandb:          train/grad_norm 0.49561
wandb:      train/learning_rate 0.0001
wandb:               train/loss 1.0617
wandb:               train_loss 1.31811
wandb:            train_runtime 749.8059
wandb: train_samples_per_second 0.3
wandb:   train_

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_

In [18]:
try:
    trainer.model.push_to_hub(new_model, use_temp_dir=False)
except:
    print("An exception occurred")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

# Step 14: Define Output Labels:

In [19]:
import re
# Define the Courses labels
courses_labels = ["Computer", "Physics", "Mathematics", "Statistics", "Quantitative Biology", "Quantitative Finance"]

In [20]:
def extract_courses(text):
    """Extracts text outside [INST] and [/INST] tags and matches against courses_labels."""
    last_inst_index = text.rfind("[/INST]")
    if last_inst_index != -1:
        my_text = text[last_inst_index + len("[/INST]"):].strip().lower()
        # Use regex to replace unwanted characters and split by whitespace
        clean_text = re.sub(r"[^\w\s]", " ", my_text)
        detected_courses = clean_text.split()

        # Debugging output (optional)
        # print('Detected Courses:', detected_courses)

        # Return a list indicating which courses are detected
        return [1 if course.lower() in detected_courses else 0 for course in courses_labels]
    else:
        return [0] * len(courses_labels)


# Step 15: Load Test Dataset 

In [21]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/kaggle/input/abstract-classification/Test_Sample.csv')

# Create a copy of the dataset
data_t = data.copy()

# Extract the 'ABSTRACT' column as test texts
test_texts = data_t['ABSTRACT'].tolist()

# Drop 'ID', 'TITLE', and 'ABSTRACT' columns to extract the actual values for emotions
test_labels = data_t.drop(columns=['ID', 'TITLE', 'ABSTRACT']).values


# Step 16: Generating Output Predictions

In [22]:
logging.set_verbosity(logging.CRITICAL)

predictions = []
for text in test_texts:
    query = f"""[INST]you are an expert who need to assess and detect courses present in given text.
            please identify and list up to four prominent courses present in the provided English text. 
            please Only Consider the following courses labels:
            Computer
            Physics
            Mathematics
            Statistics
            Quantitative Biology
            Quantitative Finance
            neutral
            

            If no courses are detected or the text is unclear, label as ['neutral'].

            your answer should be in the form of a "Python List" containing only available courses labels.

            you will only respond with the label given. Do not include the word "label". Do not provide explanation of notes.

        The text is : \"{text}\" [/INST] Courses: """
    
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=20 )
    result = pipe(query)
    print(result[0]['generated_text'])
    #result = llm(query)
    #display(Markdown(f"<b>{query.removeprefix('[INST]').removesuffix('[/INST]')}</b>"))
    #display(Markdown(f"<p>{result[0]['generated_text']}</p>"))
    r_label = extract_courses(result[0]['generated_text'])
    print("****", r_label)
    predictions.append(r_label)

[INST]you are an expert who need to assess and detect courses present in given text.
            please identify and list up to four prominent courses present in the provided English text. 
            please Only Consider the following courses labels:
            Computer
            Physics
            Mathematics
            Statistics
            Quantitative Biology
            Quantitative Finance
            neutral
            

            If no courses are detected or the text is unclear, label as ['neutral'].

            your answer should be in the form of a "Python List" containing only available courses labels.

            you will only respond with the label given. Do not include the word "label". Do not provide explanation of notes.

        The text is : "  We consider the problem of estimating the $L_1$ distance between two discrete
probability measures $P$ and $Q$ from empirical data in a nonasymptotic and
large alphabet setting. When $Q$ is known and one obtains $

# Step 17: DataFrames for Predicted and Actual Output Labels

## Step 17.1: Convert into DataFrame

In [23]:
results_df = pd.DataFrame(predictions, columns=courses_labels)
print(results_df)
actual_df = pd.DataFrame(test_labels, columns=courses_labels)
print(actual_df)

    Computer  Physics  Mathematics  Statistics  Quantitative Biology  \
0          0        0            1           1                     0   
1          0        0            0           0                     0   
2          0        0            1           1                     0   
3          0        0            0           0                     0   
4          1        0            1           0                     0   
5          0        0            1           1                     0   
6          0        0            1           0                     0   
7          0        0            0           0                     0   
8          0        0            0           0                     0   
9          0        0            1           0                     0   
10         0        0            0           0                     0   
11         1        0            1           0                     0   
12         0        0            1           1                  

## Step 17.2: Comparing Actual and Predicted Course Labels for Sample Data

In [24]:
for i in range(20):
    print(test_labels[i],"\n",predictions[i], "\n\n")
#print(test_labels, "\n\n", predictions)

[0 0 1 1 0 0] 
 [0, 0, 1, 1, 0, 0] 


[0 1 1 0 0 0] 
 [0, 0, 0, 0, 0, 0] 


[1 0 0 1 0 0] 
 [0, 0, 1, 1, 0, 0] 


[0 0 1 0 0 0] 
 [0, 0, 0, 0, 0, 0] 


[1 0 0 0 0 0] 
 [1, 0, 1, 0, 0, 0] 


[1 0 1 0 0 0] 
 [0, 0, 1, 1, 0, 0] 


[0 1 0 0 0 0] 
 [0, 0, 1, 0, 0, 0] 


[0 1 0 0 0 0] 
 [0, 0, 0, 0, 0, 0] 


[1 0 1 0 0 0] 
 [0, 0, 0, 0, 0, 0] 


[1 0 0 0 0 0] 
 [0, 0, 1, 0, 0, 0] 


[0 1 0 0 0 0] 
 [0, 0, 0, 0, 0, 0] 


[1 0 0 0 0 0] 
 [1, 0, 1, 0, 0, 0] 


[1 0 0 1 0 0] 
 [0, 0, 1, 1, 0, 0] 


[1 0 0 0 0 0] 
 [0, 0, 1, 1, 0, 0] 


[0 0 1 0 0 0] 
 [0, 0, 0, 0, 0, 0] 


[1 0 0 1 0 0] 
 [1, 0, 1, 0, 0, 0] 


[0 0 1 0 0 0] 
 [0, 0, 1, 0, 0, 0] 


[0 0 1 1 0 0] 
 [0, 0, 0, 0, 0, 0] 


[0 1 0 0 0 0] 
 [0, 0, 0, 0, 0, 0] 


[1 0 1 1 0 0] 
 [0, 0, 1, 0, 0, 0] 




# Step 18: Evaluation Metrics for Output Label Predictions

In [25]:
from sklearn.metrics import f1_score, precision_score, recall_score
# Compute evaluation metrics
f1_micro = f1_score(test_labels, predictions, average='micro')
f1_macro = f1_score(test_labels, predictions, average='macro')
precision_micro = precision_score(test_labels, predictions, average='micro')
precision_macro = precision_score(test_labels, predictions, average='macro')
recall_micro = recall_score(test_labels, predictions, average='micro')
recall_macro = recall_score(test_labels, predictions, average='macro')

# Print the evaluation metrics
print(f"F1 Micro: {f1_micro}")
print(f"F1 Macro: {f1_macro}")
print(f"Precision Micro: {precision_micro}")
print(f"Precision Macro: {precision_macro}")
print(f"Recall Micro: {recall_micro}")
print(f"Recall Macro: {recall_macro}")

F1 Micro: 0.3571428571428571
F1 Macro: 0.2127329192546584
Precision Micro: 0.47619047619047616
Precision Macro: 0.31794871794871793
Recall Micro: 0.2857142857142857
Recall Macro: 0.18354978354978355


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<br><br><br>
<center> <h3 style="color:blue">==========================================================</h3> </center>
<center> <h3 style="color:green">JAZAK ALLAH KHAIR</h3> 
<center> <h3 style="color:blue">==========================================================</h3> </center>
<br><br><br>